# fakefriends.csv
- user_id
- ID name
- age
- num_friends

# 분석
1. 평균 친구수 구하기
2. 가장 친구가 많은 사용자 찾기
3. 친구 수가 100 이상인 사용자 필터링
4. 특정 나이에 속한 사람들의 친구 수 합계

In [1]:
from pyspark import SparkConf, SparkContext
from collections import OrderedDict
conf = SparkConf().setMaster("local").setAppName("fake-friends")
sc = SparkContext(conf=conf)

24/12/05 15:26:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
import os
directory = os.path.join(os.getcwd(), "data") 
filename  = "fakefriends.csv"
file_path = os.path.join(directory, filename)
lines = sc.textFile("file:///" + file_path.replace("\\", "/"))

In [3]:
lines.first()

'0,Will,33,385'

In [17]:
lines.take(5)

['0,Will,33,385',
 '1,Jean-Luc,26,2',
 '2,Hugh,55,221',
 '3,Deanna,40,465',
 '4,Quark,68,21']

In [4]:
def parse(line):
    fields = line.split(',')
    user_id = int(fields[0])
    name = fields[1]
    age = int(fields[2])
    num_friends = int(fields[3])
    return user_id, name, age, num_friends

In [7]:
# 평균 친구 수 
data = lines.map(parse)

total_friends = data.map(lambda x:x[3]).sum()
num_users = data.count()
average_friends = total_friends / num_users
print(average_friends)

248.532


In [8]:
# 가장 친구가 많은 사용자
max_friends = data.max(key=lambda x:x[3])
print(f"id가 {max_friends[0]}이고 이름이 {max_friends[1]}인 사용자가 친구가 {max_friends[3]}명으로 가장 많음")

id가 106이고 이름이 Beverly인 사용자가 친구가 499명으로 가장 많음


In [10]:
# 친구 수가 100 이상인 사용자 필터링
over_100_friends = data.filter(lambda x:x[3]>= 100).collect()
for i in over_100_friends:
    print(f"id는 {i[0]}이고 이름은 {i[1]}인 사용자는 친구가 {i[3]}명임")

id는 0이고 이름은 Will인 사용자는 친구가 385명임
id는 2이고 이름은 Hugh인 사용자는 친구가 221명임
id는 3이고 이름은 Deanna인 사용자는 친구가 465명임
id는 5이고 이름은 Weyoun인 사용자는 친구가 318명임
id는 6이고 이름은 Gowron인 사용자는 친구가 220명임
id는 7이고 이름은 Will인 사용자는 친구가 307명임
id는 8이고 이름은 Jadzia인 사용자는 친구가 380명임
id는 9이고 이름은 Hugh인 사용자는 친구가 181명임
id는 10이고 이름은 Odo인 사용자는 친구가 191명임
id는 11이고 이름은 Ben인 사용자는 친구가 372명임
id는 12이고 이름은 Keiko인 사용자는 친구가 253명임
id는 13이고 이름은 Jean-Luc인 사용자는 친구가 444명임
id는 16이고 이름은 Weyoun인 사용자는 친구가 323명임
id는 18이고 이름은 Jean-Luc인 사용자는 친구가 455명임
id는 19이고 이름은 Geordi인 사용자는 친구가 246명임
id는 20이고 이름은 Odo인 사용자는 친구가 220명임
id는 21이고 이름은 Miles인 사용자는 친구가 268명임
id는 23이고 이름은 Keiko인 사용자는 친구가 271명임
id는 25이고 이름은 Ben인 사용자는 친구가 445명임
id는 26이고 이름은 Julian인 사용자는 친구가 100명임
id는 27이고 이름은 Leeta인 사용자는 친구가 363명임
id는 28이고 이름은 Martok인 사용자는 친구가 476명임
id는 29이고 이름은 Nog인 사용자는 친구가 364명임
id는 30이고 이름은 Keiko인 사용자는 친구가 175명임
id는 31이고 이름은 Miles인 사용자는 친구가 161명임
id는 32이고 이름은 Nog인 사용자는 친구가 281명임
id는 33이고 이름은 Dukat인 사용자는 친구가 197명임
id는 34이고 이름은 Jean-Luc인 사용자는 친구가 249명임
id는 35이고 이름은 Beverly인 사용자는 

In [14]:
# 나이대
age_distribution = (
    data.map(lambda x:(x[2]//10*10, 1)).reduceByKey(lambda x,y:x+y).sortByKey()
)
    
for age, count in age_distribution.collect():
    print(f"{age}대 : {count}명")

10대 : 19명
20대 : 93명
30대 : 97명
40대 : 102명
50대 : 94명
60대 : 95명


In [16]:
# 나이대별 속한 사람들의 친구 수 합계
def age_group_num_friends(age_group):
    num_friends = data.filter(lambda x:x[2]//10*10==age_group).map(lambda x:x[3]).sum()
    return num_friends
    

age_10 = age_group_num_friends(10)
print(f"나이가 10대인 사람의 친구수는 {age_10}명")
age_20 = age_group_num_friends(20)
print(f"나이가 20대인 사람의 친구수는 {age_20}명")
age_30 = age_group_num_friends(30)
print(f"나이가 30대인 사람의 친구수는 {age_30}명")
age_40 = age_group_num_friends(40)
print(f"나이가 40대인 사람의 친구수는 {age_40}명")
age_50 = age_group_num_friends(50)
print(f"나이가 50대인 사람의 친구수는 {age_50}명")
age_60 = age_group_num_friends(60)
print(f"나이가 60대인 사람의 친구수는 {age_60}명")

나이가 10대인 사람의 친구수는 5093명
나이가 20대인 사람의 친구수는 21503명
나이가 30대인 사람의 친구수는 22887명
나이가 40대인 사람의 친구수는 26455명
나이가 50대인 사람의 친구수는 24360명
나이가 60대인 사람의 친구수는 23968명


In [18]:
sc.stop()